In [97]:
#import libraries 
from bs4 import BeautifulSoup
import requests
import time
import datetime
import csv
import pandas as pd

import smtplib, ssl

In [23]:
#Connect to the website 

URL = 'https://www.amazon.com/Gildan-Mens-T-Shirt-Assortment-X-Large/dp/B077ZJXCTS/ref=sr_1_2?crid=AYMNYPTPMGCC&keywords=t-shirt&qid=1687426203&sprefix=tshir%2Caps%2C193&sr=8-2'

headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

page = requests.get(URL, headers=headers)

soup1 = BeautifulSoup(page.content, "html.parser")

soup2 = BeautifulSoup(soup1.prettify(),"html.parser")

#get title

title = soup2.find(id='productTitle').get_text()

#get price (a bit moire complicated since the id changed lately)

def get_price(soup):

    try:
         price = soup.find("span", attrs={'class': 'a-offscreen'}).string.strip().replace(',', '')

    except AttributeError:
        price = ""

    return price



price = get_price(soup2)

print (price)
print (title)

$19.99

                   Gildan Men's Crew T-Shirts, Multipack, Style G1100
                  


In [36]:
#remove the $ sign from price 

price = price.strip()[1:]

#remove the blank spaces from title
title = title.strip()


print(title)
print(price)

Gildan Men's Crew T-Shirts, Multipack, Style G1100
19.99


In [53]:
#put timestamp

today = datetime.date.today()

print(today)

2023-06-22


In [54]:
#create and import in a csv file 

header = ['Title','Price','Date']
data = [title,price,today]

#with open('AmazonWebScraperDataset.csv', 'w', newline='', encoding='UTF8') as f:
   # writer = csv.writer(f)
   # writer.writerow(header)
    #writer.writerow(data)

In [57]:
#read the csv file 

df = pd.read_csv('/Users/yacinelandolsi/Introduction Python Youtube/Pandas Tutorials/Web scraping project/AmazonWebScraperDataset.csv')
df

,Title,Price,Date
0,"Gildan Men's Crew T-Shirts, Multipack, Style G...",19.99,2023-06-22
1,"Gildan Men's Crew T-Shirts, Multipack, Style G...",19.99,2023-06-22


In [56]:
#Appendgin data to csv

with open('AmazonWebScraperDataset.csv', 'a+', newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(data)

In [118]:
#Create the function

def check_price():
    
    URL = 'https://www.amazon.com/Gildan-Mens-T-Shirt-Assortment-X-Large/dp/B077ZJXCTS/ref=sr_1_2?crid=AYMNYPTPMGCC&keywords=t-shirt&qid=1687426203&sprefix=tshir%2Caps%2C193&sr=8-2'

    headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    page = requests.get(URL, headers=headers)

    soup1 = BeautifulSoup(page.content, "html.parser")

    soup2 = BeautifulSoup(soup1.prettify(),"html.parser")

    #get title

    title = soup2.find(id='productTitle').get_text()

    #get price (a bit moire complicated since the id changed lately)

    def get_price(soup):

        try:
             price = soup.find("span", attrs={'class': 'a-offscreen'}).string.strip().replace(',', '')

        except AttributeError:
            price = ""

        return price

    price = get_price(soup2)
    
    #remove blank space and $
    
    price = price.strip()[1:]

    title = title.strip()

    #put timestamp

    today = datetime.date.today()
    
    #import/add to csv file
    
    header = ['Title','Price','Date']
    data = [title,price,today]

    with open('AmazonWebScraperDataset.csv', 'a+', newline='', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(data)
      
    #send an email if the price is below 14
    
    if(float(price) < 20):
        send_email()
        
    

In [85]:
#automatised the process

while(True):
    check_price()
    time.sleep(86400)

KeyboardInterrupt: 

In [115]:
#send en email alert when price reach a certain price

import smtplib, ssl
from email.message import EmailMessage

def send_email():
    port = 465  # For SSL
    smtp_server = "smtp.gmail.com"
    sender_email = "yacine.landolsi@gmail.com"  # Enter your address
    receiver_email = "yacine.landolsi1@gmail.com"  # Enter receiver address
    password = "vtftqxpafoviuabm"

    msg = EmailMessage()
    msg.set_content("Yacine, This is the moment we have been waiting for. Now is your chance to pick up the shirt of your dreams. Don't mess it up! Link here: https://www.amazon.com/Funny-Data-Systems-Business-Analyst/dp/B07FNW9FGJ/ref=sr_1_3?dchild=1&keywords=data+analyst+tshirt&qid=1626655184&sr=8-3")
    msg['Subject'] = "The Shirt you want is below $20! Now is your chance to buy!"
    msg['From'] = sender_email
    msg['To'] = receiver_email

    context = ssl.create_default_context()
    with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
        server.login(sender_email, password)
        server.send_message(msg, from_addr=sender_email, to_addrs=receiver_email)

In [114]:
send_email()

In [119]:
check_price()